In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
from sklearn import datasets, linear_model, utils, preprocessing
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns

# Linear and Logistic Regression: What this is all about
<!-- requirement: images/linear_regression_error.gif -->

*&copy; The Data Incubator*

You might be familiar with term *linear regression*. It is a mathematical technique for identifying linear relationships in data.

We are going to explore Linear Regression using a data set called the *California Housing Data*. This is a useful data set for building and benchmarking models. The data set contains aggregated data on housing values and characteristics by census block. Researchers can study the data to predict the median value of homes based on the other variables.

This is a standard data set and comes with Scikit Learn

In [ ]:
from sklearn.datasets import fetch_california_housing
cali_data = fetch_california_housing()
X = cali_data['data']
y = cali_data['target']
print(cali_data['DESCR'])

Let's add the data to a DataFrame to make it easier to use.

In [ ]:
names = cali_data['feature_names']

data_dict = dict(zip(names, ['Median income', 'House age', 'Average # of rooms', 'Average # of bedrooms', 'Population', 'Average occupancy', 'Latitude', 'Longitude']))

cali_df = DataFrame(cali_data['data'], columns=names)

home_values = Series(cali_data['target'])

In [ ]:
cali_df.head()

In [ ]:
home_values.head()

We can familiarize ourselves with the data by plotting it. Let's have some fun exploring data using `IPython` widgets.

Experiment with the dropdown to plot each column vs. Median home value.

**Question:** Which columns seem to have somewhat of a linear relationship with home values?

In [ ]:
from ipywidgets import widgets

def cali_plot(column):
    plt.plot(cali_df[column], home_values, '.')
    plt.xlabel(data_dict[column])
    plt.ylabel('Home Price')

dropdown_values = {"{0}: {1}".format(k, v):k for k, v in data_dict.items()}

widgets.interact(cali_plot, column=dropdown_values);

The median income feature in particular seems to have a linear relationship with home price. We can attempt to visualize that by adding a straight line to the chart.

**Question:** In the below figure, which line seems to best "fit" the data?

In [ ]:
plt.plot(cali_df['MedInc'], home_values, '.')
plt.plot([0, 9], [0, 5], '-', color='darkorange')
plt.plot([.5, 8], [0, 5], '-', color='limegreen',)
plt.plot([2, 5], [0, 5], '-', color='red')

plt.xlabel(data_dict['MedInc'])
plt.ylabel('Home Price');

The red one looks like a poor fit to me but I can't decide if the green or orange one is the better.

But rather than eye-balling lines on charts, wouldn't it be great if there was a way to pick the "best" line that fits  the data?

Linear regression is just that process. It is a mathematical process for measuring a line's "error" with respect to the data and picking the line that minimizes that error. It can describe the linear relationship between a set of numbers (the independent variables) and the dependent variable.

In this case the independent variable is the "average number of rooms per dwelling" and the dependent variable is the Home Price. The independent variables need not be limited to one variable. A complex model may have hundreds or more dependent variables!

Let's try Linear Regression to find the best fitting line. Is this result what you expected?

In [ ]:
linreg = linear_model.LinearRegression(fit_intercept=True)  # fit_intercept=True is the default value
linreg.fit(cali_df[['MedInc']], home_values)
x = np.linspace(-1, 15).reshape(-1,1)

plt.plot(cali_df['MedInc'], home_values, '.')
plt.plot(x, linreg.predict(x), '-')
plt.ylim(0, 5)
plt.xlim(0, 15)
plt.xlabel(data_dict['MedInc'])
plt.ylabel('Home Price');

Let's dig deeper into how linear regression works.

## Linear Regression


This is the basic picture of linear regression errors:

![$L^1$ versus $L^2$ regularization](images/linear_regression_error.gif)

Linear Regression is perhaps the simplest linear model $f(X_{j \cdot}) = \sum_i \beta_i X_{ji}$.  The error model assumes the $y_j$'s are independent and normally distributed around $X_{ji} \cdot \beta_i$ with standard deviation $\sigma$.

### Likelihood and cost functions

Suppose that we knew that the correct model was given by some $\beta_i$.  Given the above assumption about the error model, the probability of measuring $y_j$ is simply

$$ P(y_j \mid \beta_i) = \prod_j \frac{1}{\sqrt{2\pi \sigma^2}} \exp \left[-\left( \frac{X_{ji} \cdot \beta_i - y_j}{2 \sigma} \right)^2 \right] \,.$$

However, we don't know the $\beta_i$.  Instead we want to find them, given the $y_j$, by finding the $\beta_i$ that maximize $P(\beta_i \mid y_j)$.  Thanks to Bayes' Rule, we know

$$ P(\beta_i \mid y_j) = P(y_j \mid \beta_i) \frac{P(\beta_i)}{P(y_j)} \,.$$

We know the first term on the right, and $P(y_j)$ is independent of $\beta_i$, leaving only $P(\beta_i)$ unknown.  In linear regression, we suppose we have no *a priori* knowledge of the expected coefficients and take $P(\beta_i)$ to be constant as well.  Thus, the most probable model is determined by maximizing the likelihood function

$$ L(\beta) = \prod_j \frac{1}{\sqrt{2\pi \sigma^2}} \exp \left[-\left( \frac{X_{ji} \cdot \beta_i - y_j}{2 \sigma} \right)^2 \right] \propto P(\beta_i \mid y_j) \,.$$

Since $\log$ is monotonic, we can also maximize the log-likelihood.  A few calculations show us that the negative log-likelihood (up to a linear transformation) is

$$- \log(L(\beta)) \sim \| y - X \beta \|^2\,.$$

Here, $\| z \| = \| z \|_2 = \sum_i |z_i|^2 $ is the $L^2$ norm.  The objective is to minimize this quadratic:

$$ \min_\beta \| y - X \beta \|^2\,.$$

Of course, this is the familiar expression for linear regression.  We could minimize $\beta$ via gradient descent, but it turns out that the solution has a closed form, 

$$ X \hat \beta = y\,, $$

or

$$ \hat \beta = (X^T X)^{-1} X^T y\,. $$

## Using Linear Regression


Let's see how this is used in practice. We will do a Linear Regression as before, but will shuffle the data first.

**Question:** Why is shuffling the data a good idea?

In [ ]:
Xraw, y = utils.shuffle(cali_df, home_values)

### Prepare the data with a train-test split


We use TRAIN data to fit our model, and we predict on TEST data.

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(Xraw, y, test_size=0.2)

Cross validation allows us to *train* the model on a subset of the data and later *test* the model on a different subset of data.

We shuffled the data to make sure that cross validation split is being done randomly. Sometimes the ordering of the data set you start out with is not random. Watch out for this!

In [ ]:
print Xraw.shape
print X_train.shape
print X_test.shape

We also want to scale our data so that it has a mean of zero and a variance of one. Scaling the data is helpful when the input variables have different magnitudes and ranges. For example in the California data set the relative scale of the `CHAS` and `NOX` columns is very different from that of the TAX column.

Scikit-learn has a transformer called `StandardScaler` that does exactly what we need.

Observe that we `fit` the `StandardScaler` with only the training data and then `transform` both the training and test data. This ensures that the `StandardScaler` is not scaling based on information from the test data.

In [ ]:
scaler = preprocessing.StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

We can validate that the mean and variance is scaled as required.

In [ ]:
train_means = X_train.mean(axis=0).to_frame('mean')

train_means['var'] = X_train.var(axis=0)
train_means['scaled_mean'] = X_train_scaled.mean(axis=0)
train_means['scaled_var'] = X_train_scaled.var(axis=0)

train_means

The test data statistics look similar but observe the `scaled_mean` values are small but not equal to zero. This is expected because the `StandardScaler` was fit with only the training data.

In [ ]:
test_means = X_test.mean(axis=0).to_frame('mean')
test_means['var'] = X_test.var(axis=0)
test_means['scaled_mean'] = X_test_scaled.mean(axis=0)
test_means['scaled_var'] = X_test_scaled.var(axis=0)

test_means

### Fit the model to the training data


Now that our data is ready, create a Scikit-learn `LinearRegression` model and fit the model to the training data.

In [ ]:
linreg = linear_model.LinearRegression()

linreg.fit(X_train_scaled, y_train)

### Use the fitted model to make predictions with the test data


Feed in the test data to make out "out-of-sample" predictions.

In [ ]:
linreg.predict(X_test_scaled)

### Model Parameters


What are the model parameters?

In [ ]:
linreg.coef_

In [ ]:
linreg.intercept_

What do those numbers mean?

They are parameters in the following equation. This equation defines the model's linear relationship between the the data attributes and the cost of homes in California.

In [ ]:
print ("prediction = " +
       "{0} +\n".format(linreg.intercept_) +
       " +\n".join(["{1} * {0}".format(n, f) for n, f in zip(names, linreg.coef_)]))

When the model is making predictions, it throws the test data in that equation and calculates the predicted value.

### Model Error Measurements


This is the $R^2$ of the fitted model on the training data.

This number can be compared to other models on the same data set.

In [ ]:
linreg.score(X_train_scaled, y_train)

And the $R^2$ of the fitted model on the test data.

In [ ]:
linreg.score(X_test_scaled, y_test)

# Regularization


Linear Regression results can be improved with Regularization techniques.

Regularization adds a "cost" to the optimization, penalizing larger coefficient values. This can help combat overfitting.

## Ridge Regression


Ridge regression adds a penalty to the Linear Regression optimization that is proportional to the sum of the squared parameter values, like this:

$$- \log(L(\beta)) \sim \| y - X \beta \|^2 + \alpha \| \beta \|^2\,.$$

This reduces the occurrence of extreme positive or negative values sometimes this improves out-of-sample model performance.

### Fit the model to the training data


Again, fit model with the training data.

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=5.0)

ridge.fit(X_train_scaled, y_train)

### Use the fitted model to make predictions with the test data

In [ ]:
ridge.predict(X_test_scaled)

### Model Parameters


What are the model parameters?

In [ ]:
ridge.coef_

Observe that the sum of the squared model parameters is smaller than before.

In [ ]:
print sum(linreg.coef_ ** 2)
print sum(ridge.coef_ ** 2)

### Model Error Measurements


The $R^2$ of the fitted model on the training data:

In [ ]:
ridge.score(X_train_scaled, y_train)

And the out-of-sample test data. This is marginally better than the first linear regression model.

In [ ]:
ridge.score(X_test_scaled, y_test)

## `RidgeCV`


`RidgeCV()` is just like `Ridge()` but with cross-validation built in.

### Fit the model to the training data

In [ ]:
from sklearn.linear_model import RidgeCV

ridgecv = RidgeCV(alphas=(0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 3.5, 5.0, 7.5, 10.0))

ridgecv.fit(X_train_scaled, y_train)

### Use the fitted model to make predictions with the test data

In [ ]:
ridgecv.predict(X_test_scaled)

### Model Parameters


What are the model parameters?

In [ ]:
ridgecv.coef_

And the `alpha` value?

In [ ]:
ridgecv.alpha_

### Model Error Measurements


The $R^2$ of the fitted model on the training data:

In [ ]:
ridgecv.score(X_train_scaled, y_train)

And the out-of-sample test data. This is marginally better than the first linear regression model.

In [ ]:
ridgecv.score(X_test_scaled, y_test)

## Lasso Regularization


Lasso is like ridge regression but has the ability to automatically select features.  The objective function to minimize is

$$ \frac{1}{2 n} \| y - X^T \beta \|^2 + \alpha \|\beta\|_1 $$

where $\|\beta\|_1 = \sum_i |\beta_i| $ is the $L^1$ norm (sum of the absolute values) of $\beta$ and $n$ is the number of samples. Lasso has a feature selection property where many weights on features are zero (i.e. those features are not selected).

### Fit the model to the training data

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.2)

lasso.fit(X_train_scaled, y_train)

### Use the fitted model to make predictions with the test data

In [ ]:
lasso.predict(X_test_scaled)

### Model Parameters


Observe that some of the parameters are zero:

In [ ]:
lasso.coef_

That means some of the attributes are essentially excluded from the model.

In [ ]:
print ("prediction = " +
       "{0} +\n".format(lasso.intercept_) +
       " +\n".join(["{1} * {0}".format(n, f) for n, f in zip(names, lasso.coef_)]))

### Model Error Measurements


The $R^2$ of the fitted model on the training data:

In [ ]:
lasso.score(X_train_scaled, y_train)

And the test data:

In [ ]:
lasso.score(X_test_scaled, y_test)

# Logistic Regression


In Logistic Regression, the values of $y$ are categorical ($0$ or $1$) and assumed to be distributed binomially.  We assume that the probability $p(X_{j\cdot})$ that $y = 1$ is related to $X$ via the logit function:

$$ \mbox{logit }(p(X_{j\cdot})) = \log \frac{p(X_{j\cdot})}{1-p(X_{j\cdot})} = X_{ji} \cdot \beta_i\,. $$

Notice that the logit function $\log \frac{x}{1-x}$ is just the log odds and maps the real numbers $[0,1]$ to $\mathbb{R}$.

Below is a plot of the logit function.

In [ ]:
lx = np.linspace(0.001,0.999,100)
ly = np.log(lx/(1-lx))
plt.plot(lx,ly)
plt.xlabel('$p$')
plt.ylabel(r'$\mathrm{logit}(p)$');

It may be more clear to invert this to get an expression for $p(X_{j\cdot})$:

$$ p(X_{j\cdot}) = \frac{\exp\left( X_{ji} \cdot \beta_i\right)}{1 + \exp\left( X_{ji} \cdot \beta_i\right)} $$

The input can vary over the entire real numbers, but the output is always a valid probability between 0 and 1.

In [ ]:
lx = np.linspace(-10, 10)
ly = np.exp(lx) / (1 + np.exp(lx))
plt.plot(lx, ly)
plt.xlabel(r'$X_{ji} \cdot \beta_i$')
plt.ylabel(r'$P(X_{j\cdot})$');

In the following example, we'll try to predict whether the home price is greater than or less than $25K.

In [ ]:
y_cat_train = y_train > 2.
y_cat_test = y_test > 2.

### Fit the model to the training data

In [ ]:
logistic = linear_model.LogisticRegression()

logistic.fit(X_train_scaled, y_cat_train)

### Use the fitted model to make predictions with the test data


Observe the predictions are now True/False values

In [ ]:
logistic.predict(X_test_scaled)

### Model Parameters

In [ ]:
logistic.coef_

### Model Error Measurements


$R^2$ doesn't make sense for logistic regression, so this uses % accuracy instead.

In [ ]:
logistic.score(X_train_scaled, y_cat_train)

In [ ]:
logistic.score(X_test_scaled, y_cat_test)

## Multiclass classification problems


So far we have talked about Two-Class classification in the context of Logistic Regression.  But what if we have more than two classes?  There are generally two strategies to "bootstrap" a binary classifier to a multi-class classifier: 
1. **One-versus-All**: For each class $k=1,\ldots,K$, build a binary classifier for all points with label $y = k$ versus $y \neq k$.
1. **All-versus-All**: For each class $k \neq k'$, construct a binary classifier to distinguish between class $k$ and $k'$.

[Scikit](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) uses One-versus-All for Multi-class Logistic Regression.

If $f_k(x)$ is the predictor for class $k$, the probability of class $k$ is just the normalized predictions,

$$ p_k = \frac{f_k(x)}{\sum_k f_k(x)}$$

Or in other words, each prediction divided by the sum of the predictions.

### Questions

1. How would you assess whether a relationship is actually linear?
1. Imagine that when you loaded your data, you unwittingly loaded each row of the data (both $X$ and $y$) twice and performed the same regression.  What is the effect on your estimates $\beta$?
1. Imagine when you loaded your data, you unwittingly loaded each column of the features (just $X$) twice and performed the same regression.  What is the effect on your estimates $\beta$?
1. Everything we've talked about so far involves loading all the data into memory.  What if you have more data than you can fit into memory?

### Exit Tickets

1. Enumerate the similarities and differences between linear regression and logistic regression.
1. What is the purpose of splitting a data set into testing and training data sets?
1. Why is it important to randomize a data set before splitting it into test and training data sets?
1. What are the benefits of regularization?

### Question Answers


1. To assess if the relationship is linear, plot the distribution of the residuals as a function of $x$.  If there's a systematic bias, take a look at it and see what's going on.
1. Loading rows twice has no effect on $\beta$ but it does artificially increase your confidence (dividing it by a factor $\sqrt{2}$)
1. The problem becomes degenerate and $\beta_j$ is now split between $\beta_{j'}$ and $\beta_{j''}$ such that $\beta_j = \beta_{j'} + \beta_{j''}$.
1. All of these problems can be solved using gradient descent, which only requires a *stream* of data, rather than the entire data set.  Linear regression (with either $L^2$, Huber penalty, epsilon insensitive) can be solved using `sklearn.linear_model.SGDRegressor` and logistic regression can be solved using `sklearn.linear_model.SGDClassifier`.  These methods implement a `partial_fit` method, which can iteratively updates the coefficients on small chunks of data.  In this case, you are no longer ram constrained, but constrained in the amount of time it takes to read data from disk.

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*